In [ ]:
import h5py
import healpy as hp

sim_dir = "/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/"

map = h5py.File(sim_dir + "./foregroundmap.h5", 'r')
print map
map.items()
map['index_map'].items()

In [ ]:
#Run these in terminal first!
#source /home/k/krs/tretyako/source/ch_pipeline_master/ch_pipeline/venv/bin/activate
#or maybe mkvenv.sh? Compare to stable.sh to be sure.
#source /home/k/krs/jrs65/chime/stable.sh


In [ ]:
import numpy as np
import pickle

nfreq = 2
full_arr = 136

alphafile = './test.txt'

tmp = np.arange(nfreq*full_arr, dtype=np.float)
tmp = tmp.reshape((nfreq, full_arr))

print tmp.shape

alpha_mat = 0.1*np.ones_like(tmp)

print alpha_mat.shape

output = open('test.pkl', 'wb')
pickle.dump(alpha_mat, output)
output.close()

pkl_file = open('test.pkl', 'rb')

alpha_mat = pickle.load(pkl_file)
#pprint.pprint(data1)

pkl_file.close()

#COMMENT OUT FOR NOW    
#np.savetxt(alphafile, alpha_mat.view(float).reshape(-1, 2))
#alpha_mat = np.loadtxt(alphafile).view(complex).reshape(-1)

print alpha_mat.shape

In [ ]:
import numpy as np
from drift.core import manager
from draco.core import io
#from draco.core import io
from draco.synthesis import stream
from caput import config
#from ch_pipeline.ch_pipeline.synthesis.crosstalk import crosstalk
from draco.synthesis import crosstalk

from ch_util import tools

lbt = io.LoadBeamTransfer()

lbt.product_directory = "/scratch/k/krs/tretyako/ch_pipeline_runs/beamtransfers/test/bt/"

tel, bt = lbt.setup()

lm = io.LoadMaps()

d = dict()
e = dict()
a = "/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/foregroundmap-new.h5"
#d.setdefault('files', []).append(a)
#e.setdefault('maps', []).append(d)

#print d
#print e



#f = {'files': ['/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/foregroundmap-new.h5']}
#f.files.items()

#maps = {'maps': [{'files': ['/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/foregroundmap-new.h5']}]}


#print maps.pop(0)

#group = maps.pop(0)

lm.maps = {'files': ['/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/foregroundmap-new.h5'], 'tags': ['test']}

skymap = lm.next()

ss = stream.SimulateSidereal()

ss.setup(bt)

sstream = ss.process(skymap)

xc = crosstalk.SimulateCrossTalk()


import numpy as np
import pickle

#nfreq = 2
#full_arr = 136

#alphafile = './test.txt'

#tmp = np.arange(nfreq*full_arr, dtype=np.float)
#tmp = tmp.reshape((nfreq, full_arr))

print sstream.vis.shape

print sstream.vis[:,:,:]

#alpha_mat = 0.1*np.ones_like(tmp)

#print alpha_mat.shape

output = open('test.pkl', 'wb')
pickle.dump(sstream.vis, output, -1)
output.close()

pkl_file = open('test.pkl', 'rb')

sstream.vis[:,:,:] = pickle.load(pkl_file)
#pprint.pprint(data1)

pkl_file.close()

print sstream.vis.shape

print sstream.vis[:,:,:]

#print sstream.index_map.keys()


#print sstream.vis.shape

#print sstream.vis[:,:,0]

#print sstream.index_map['prod']


#print sstream.vis[:,:,0].shape

print tools.cmap(9, 8, 16)

for visidx in enumerate(sstream.index_map['prod']):
    print visidx

    
    

print sstream.index_map['prod'].tolist().index([9,8])

#print sstream.vis[:,:,:]

store = np.zeros_like(sstream.vis[:,:,:])

print sstream.vis[:,0,0]

mat_test = np.ones_like(sstream.vis[:,0,0])
mat = sstream.vis[:,0,:]

print mat_test
print mat

print mat_test.shape
print mat.shape

mat_test = mat_test[:, np.newaxis]

print mat_test * 2

print mat_test

print np.conjugate(mat_test) * mat

store[:,0,:] += np.conjugate(mat_test) * mat

print store[:,0,:]

#print sstream.vis[:,:,0] + 1

#print sstream.index_map['prod'].shape

#print sstream.index_map['prod']

#mat = sstream.index_map['prod']

#print sstream.index_map['prod'][[1,0]]

#print [1,0] in mat.tolist()

#print np.transpose(mat)

#mat_tran = np.transpose(mat)

#print mat[0,:]
#print mat[1,:]

#for i in mat_tran[0,:]:
#    print i
#    for j in mat_tran[1,:]:
#        print j

xc.setup(bt)

xstream = xc.process(sstream)

print 'done'


In [20]:
import numpy as np
from drift.core import manager
from draco.core import io
#from draco.core import io
from draco.synthesis import stream
from caput import config
#from ch_pipeline.ch_pipeline.synthesis.crosstalk import crosstalk
from draco.synthesis import crosstalk
from draco.analysis import transform
from draco.analysis import mapmaker

import h5py
import healpy as hp


from ch_util import tools

lbt = io.LoadBeamTransfer()

lbt.product_directory = "/scratch/k/krs/tretyako/ch_pipeline_runs/beamtransfers/test/bt/"

tel, bt = lbt.setup()

lm = io.LoadMaps()

lm.maps = {'files': ['/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/foregroundmap-new.h5'], 'tags': ['test']}

skymap = lm.next()

ss = stream.SimulateSidereal()

ss.setup(bt)

sstream = ss.process(skymap)

xc = crosstalk.SimulateCrossTalk()

xc.setup(bt)

xstream = xc.process(sstream)

mmt = transform.MModeTransform()

mmodes = mmt.process(xstream)

wmm = mapmaker.WienerMapMaker()

wmm.nside = 128
wmm.prior_tilt = 2.0
wmm.prior_amp = 50.0
wmm.save = 'yes'
wmm.output_root = '/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/map_wiener_ian_test.h5'

wmm.setup(bt)

wiener = wmm.process(mmodes)

#wiener.name = '/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/map_wiener_ian_test.h5'

print wiener.items()
print wiener['map'][0,0,:]
print wiener.name
wiener.save('/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/map_wiener_ian_test_new.h5')


#map = h5py.File("/scratch/k/krs/tretyako/ch_pipeline_runs/testsims/foregroundmap.h5", 'r')
#wiener.items()
#wiener['index_map'].items()
#hp.mollview(map['map'][0, 0])
#hp.mollview(map['map'][0, 0], min=0, max=200)
#hp.mollview(map['map'][0, 0], min=0, max=50)
#hp.mollview(map['map'][0, 0], min=0, max=50, xsize=2000)
#hp.mollview(wiener['map'][0, 0], min=0, max=50, xsize=4000)
#hp.mollview(map['map'][0, 0], min=0, max=50, xsize=4000)


print 'done'



Attempting to read telescope from disk...
[('map', <memh5 distributed dataset '/map': global_shape (2, 4, 196608), dist_axis 0, type "dtype('float64')">)]
[  1.34204055e-06   1.04893325e-06   1.63034908e-06 ...,  -2.83603231e-08
  -3.41531885e-08  -3.61932536e-08]
/
done


In [ ]:
pipeline:
    tasks:
        -   type:       draco.core.io.LoadBeamTransfer
            out:        [tel, bt]
            params:
                product_directory:  /scratch/k/krs/tretyako/ch_pipeline_runs/beamtransfers/test/bt/

        -   type:       draco.core.io.LoadMaps
            out:        skymap
            params:
                maps:
                    -    files:
                             -    'foregroundmap-new.h5'

        -   type:       draco.synthesis.stream.SimulateSidereal
            requires:   [bt]
            in:         skymap
            out:        sstream

        -   type:       draco.analysis.transform.MModeTransform
            in:         sstream
            out:        mmodes

        -   type:       draco.analysis.mapmaker.WienerMapMaker
            requires:   bt
            in:         mmodes
            out:        wiener
            params:
                nside:      128
                prior_tilt:   2.0
                prior_amp:    50.0
                save:   Yes
                output_root: map_wiener_test

